### Decorator Application: Single Dispatch Generic Functions (Part 2)

In [1]:
from decimal import Decimal
from html import escape

In [2]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{round(a, 2):.2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>'
                for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{html_escape(k)}={htmlize(v)}</li>'
                for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)

In [3]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict,
    }
    fn = registry.get(type(arg), registry[object])
    return fn(arg)

In [7]:
def singledispatch(fn):
    registry = {}

    registry[object] = fn
    def inner(arg): # <- single dispatch. Assuming the function we are decorating only takes a single argument
        return registry[object](arg)

    return inner

In [8]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [9]:
htmlize('1 < 100')

'1 &lt; 100'

In [10]:
def singledispatch(fn):
    registry = {}

    registry[object] = fn
    registry[int] = lambda a: f'{a}(<i>{str(hex(a))}</i>)'
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')

    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)

    return inner

In [11]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [14]:
htmlize('1 < 100')

'1 &lt; 100'

In [13]:
htmlize(100)

'100(<i>0x64</i>)'

In [45]:
def singledispatch(fn):
    registry = {}

    registry[object] = fn

    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner

    def dispatch(type_):
        return registry.get(type_, registry[object])

    decorated.register = register
    # decorated.registry = registry
    decorated.dispatch = dispatch

    return decorated

In [46]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [40]:
htmlize('1 < 100')

'1 &lt; 100'

In [31]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

In [48]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [29]:
htmlize(100)

'100(<i>0x64</i>)'

In [49]:
@htmlize.register(list)
def html_list(l):
    items = (f'<li>{htmlize(item)}</li>'
                for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [50]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>'
                for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [34]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [35]:
htmlize((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [44]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_sequence(l)>,
 tuple: <function __main__.html_sequence(l)>}

In [51]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [55]:
htmlize.dispatch(tuple)

<function __main__.html_sequence(l)>

In [56]:
from numbers import Integral

In [57]:
class Person:
    pass

In [59]:
class Student(Person):
    pass

In [60]:
p = Student()

In [61]:
type(p)

__main__.Student

In [62]:
isinstance(p, Student)

True

In [63]:
isinstance(p, Person)

True

In [64]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [65]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [66]:
isinstance(10, Integral)

True

In [67]:
isinstance(True, Integral)

True

In [68]:
htmlize(10)

'10'

In [69]:
@htmlize.register(int)
@htmlize.register(bool)
def html_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [70]:
htmlize(10)

'10(<i>0xa</i>)'

In [71]:
htmlize(True)

'True(<i>0x1</i>)'

In [72]:
from collections.abc import Sequence

In [73]:
isinstance([1,2,3], Sequence)

True

In [74]:
isinstance((1, 2, 3), Sequence)

True

In [75]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>'
                for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [76]:
type([1, 2, 3]) is Sequence

False

In [77]:
type([1, 2, 3])

list

In [78]:
isinstance([1, 2, 3], Sequence)

True